<!--<badge>--><a href="https://colab.research.google.com/github/ankur-98/BERT_GLUE/blob/main/multi_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# For colab run:

In [1]:
# Switch to GPU runtime
! git clone https://github.com/ankur-98/BERT_GLUE.git
import os 
os.chdir("BERT_GLUE")
! pip install datasets transformers

Cloning into 'BERT_GLUE'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 228 (delta 145), reused 123 (delta 69), pack-reused 0
Receiving objects: 100% (228/228), 152.39 KiB | 10.16 MiB/s, done.
Resolving deltas: 100% (145/145), done.
     |████████████████████████████████| 225kB 31.6MB/s 
     |████████████████████████████████| 2.3MB 38.2MB/s 
     |████████████████████████████████| 245kB 48.9MB/s 
     |████████████████████████████████| 112kB 58.3MB/s 
     |████████████████████████████████| 901kB 43.3MB/s 
     |████████████████████████████████| 3.3MB 50.0MB/s 


# Imports

In [2]:
import torch
from tqdm.auto import tqdm
from dataloader import get_dataloader
from transformers import BertModel
from model import BERTClassifierModel
from train import training_step, eval_step
from util import *

# Configs
### Tasks: {"cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"}

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_checkpoint="bert-base-uncased"
tasks = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
batch_size=4
steps = 2000
lr = 2e-5
lr_scheduler_type = "linear" # "linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"

# Load Dataloader and Pre-trained BERT Model

In [4]:
num_labels = [3 if task.startswith("mnli") else 1 if task=="stsb" else 2 for task in tasks]
train_epoch_iterator = [get_dataloader(task, model_checkpoint, "train", batch_size=batch_size) for task in tasks]
eval_epoch_iterator = [get_dataloader(task, model_checkpoint, "validation", batch_size=batch_size) for task in tasks]
BERT_model = BertModel.from_pretrained(model_checkpoint)
models = [BERTClassifierModel(BERT_model, num_labels=num_label, task=task).to(device) for num_label, task in zip(num_labels, tasks)]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.



DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence', 'token_type_ids'],
        num_rows: 1063
    })
})


Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.



DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise', 'token_type_ids'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise', 'token_type_ids'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise', 'token_type_ids'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise', 'token_type_ids'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['attention_mask', 'hypothesis', 'idx', 'input_ids', 'label', 'premise', 'token_type_ids'],
        num_rows: 9847
    })
})


Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


...

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/wnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)



DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 635
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 71
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
        num_rows: 146
    })
})


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Couldn't find file locally at matthews_correlation/matthews_correlation.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.6.2/metrics/matthews_correlation/matthews_correlation.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/metrics/matthews_correlation/matthews_correlation.py.


Couldn't find file locally at pearsonr/pearsonr.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.6.2/metrics/pearsonr/pearsonr.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/metrics/pearsonr/pearsonr.py.


# Optimizer and LR Scheduler

In [5]:
Optimizers = [create_optimizer(model, learning_rate=lr) for model in models]
LR_schedulers = [create_scheduler(Optimizer, lr_scheduler_type) for Optimizer in Optimizers]
tr_loss = []
eval_loss = []
tr_metrics = [[] for task in tasks]
eval_metrics = [[] for task in tasks]
tr_metrics_1 = [[] for task in tasks]
eval_metrics_1 = [[] for task in tasks]

# Training loop

In [6]:
global_steps = 0
steps_in_dataloaders = [len(iterator) for iterator in train_epoch_iterator]
trange = range(max(steps_in_dataloaders))
loss_pbar = tqdm(trange, initial=global_steps, total=steps, desc=f"step summary:")
pbars = [tqdm(trange, initial=global_steps, total=steps, desc=f"{task}") for task in tasks]
for e in range((steps//max(steps_in_dataloaders))+1):
    iterators = [iter(train_epoch_iterator[i]) for i in range(len(tasks))]
    for step in trange:
        global_steps += 1
        step_loss = 0
        for i, task in enumerate(tasks):
            iterator = iterators[i]
            try:
                inputs = prepare_inputs(iterator.next(), device)
            except StopIteration:
                iterators[i] = iter(train_epoch_iterator[i])
                continue
            pbars[i].update()

            step_task_loss, step_metric, step_metric_1 = training_step(models[i], inputs, 
                                                                  Optimizers[i], LR_schedulers[i])
            
            step_loss += step_task_loss
            tr_metrics[i].append(torch.tensor(list(step_metric.values())[0]))
            if models[i].metric_1 is not None: tr_metrics_1[i].append(torch.tensor(list(step_metric_1.values())[0]))
            
            step_evaluation = {}
            step_evaluation[f"{models[i].metric.__class__.__name__}"] = torch.stack(tr_metrics[i])[-steps_in_dataloaders[i]:].mean().item()
            if models[i].metric_1 is not None:
                step_evaluation[f"{models[i].metric_1.__class__.__name__}"] = torch.stack(tr_metrics_1[i])[-steps_in_dataloaders[i]:].mean().item()
            pbars[i].set_postfix(step_evaluation)
        loss_pbar.update()
        tr_loss.append(step_loss)
        step_evaluation = {}
        step_evaluation['loss'] = tr_loss[-1].mean().item()
        loss_pbar.set_postfix(step_evaluation)
        
        if global_steps == steps:
            break

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "tr

# Evaluation

In [7]:
print(f"Evaluation begins in batches of {batch_size}..")
trange = [len(iterator) for iterator in eval_epoch_iterator]
loss_pbar = tqdm(sum(trange), desc=f"step summary:")
pbars = [tqdm(range(trange[i]), desc=f"{task}") for i, task in enumerate(tasks)]
for i, task in enumerate(tasks):
    iterator = iter(eval_epoch_iterator[i])
    step_loss = 0
    for step in range(trange[i]):
        pbars[i].update()
        
        inputs = prepare_inputs(iterator.next(), device)
        step_task_loss, step_metric, step_metric_1 = eval_step(models[i], inputs)

        step_loss += step_task_loss/trange[i]
        eval_metrics[i].append(torch.tensor(list(step_metric.values())[0]))
        if models[i].metric_1 is not None: eval_metrics_1[i].append(torch.tensor(list(step_metric_1.values())[0]))
        
        step_evaluation = {}
        step_evaluation[f"{models[i].metric.__class__.__name__}"] = torch.stack(eval_metrics[i]).mean().item()
        if models[i].metric_1 is not None:
            step_evaluation[f"{models[i].metric_1.__class__.__name__}"] = torch.stack(eval_metrics_1[i]).mean().item()
        pbars[i].set_postfix(step_evaluation)
    loss_pbar.update()
    eval_loss.append(step_loss)
    step_evaluation = {}
    step_evaluation['loss'] = torch.stack(eval_loss).sum().item()
    loss_pbar.set_postfix(step_evaluation)

Evaluation begins in batches of 4..


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
 